### Обработка данных из статистических сборников Росстата
Информация в сборниках хранится в docx файлах. Внутри этих файлов находятся таблицы с данными.
Главная проблема - надкатегории и подкатегории.



In [471]:
from docx import Document
import pandas as pd
import os

In [472]:
os.chdir("/Users/yuliyakuzmina/Downloads/rosstat")

Я написала функции, которые работают с этими косяками в таблицах. 

In [473]:
def parse_sub_category_table(doc):
    groups = []
    data_frame = pd.DataFrame()
    is_next_table = False
    for table in doc.tables:
        df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    df[i][j] = cell.text
                    if j == 1 and any(c.isalpha() for c in cell.text): # поиск текстовых значений в строках 
                        groups.append(cell.text)
        df = pd.DataFrame(df)
        if is_next_table: 
            df = df.iloc[1:, :] #убираем первую строку с каждой новой страницы docx документа
        is_next_table = True
        data_frame = data_frame.append(df, ignore_index=True)
        #data_frame.columns = data_frame.iloc[0] #чтобы разделить датафрейм нужно задать названия переменных
    print(groups)
    print(data_frame.iloc[-1, -1])
    sub_dfs = []
    # разделим таблицу по подпараметрам
    if len(groups) > 1:
        for group in range(len(groups)):
            next_group = group + 1
            # неудачная попытка разделить датафрейм через индексы:
            sub_dfs.append(data_frame[data_frame[data_frame.iloc[:,1]==groups[group]].index.values.astype(int)[0] \
         : data_frame[data_frame.iloc[:,1]==groups[next_group]].index.values.astype(int)[0]])
            
            sub_dfs[group].to_csv(time.strftime("%Y%m%d-%H%M%S") + '.csv')
    else:
        data_frame.to_csv(time.strftime("%Y%m%d-%H%M%S") + '.csv')

        
def parse_two_column_table(doc):
    groups = set()    
    data_frame = pd.DataFrame()
    is_next_table = False
    for table in doc.tables:
        df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    df[i][j] = cell.text
                if i == 0 and cell.text: # поиск уникальных значений в строке с "всего" и " на 10 000 человек" и пр.
                    groups.add(cell.text)
        df = pd.DataFrame(df)
        if is_next_table:
            df = df.iloc[2:, :]
        is_next_table = True
        data_frame = data_frame.append(df, ignore_index=True)
    print(groups)
    print(data_frame.iloc[-1, -1])
    
    sub_dfs = []
    # разделим таблицу по группам 
    if len(groups) > 1:
        for group_name in list(groups):
            col_filter = data_frame.iloc[0, :] == group_name
            col_filter[0] = True
            sub_dfs.append(data_frame.loc[:, col_filter])
    else:
        sub_dfs.append(data_frame)
    #сохраняем
    sub_dfs[0].to_csv(time.strftime("%Y%m%d-%H%M%S") + 'a.csv')
    sub_dfs[1].to_csv(time.strftime("%Y%m%d-%H%M%S") +'b.csv')

    
def parse_docx(file_name):
    file_path = "%s" % file_name
    doc = Document(file_name)
    print(doc.paragraphs[0].text)
    if doc.tables[0].rows[0].cells[1].text.isnumeric(): # проверка на наличие над-категорий
        parse_sub_category_table(doc)
    else:
        parse_two_column_table(doc)

Я выбрала несколько показателей и предварительно сохранила их себе.

Список показателей:
- [Число больничных коек](https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-01.docx)  (в таблице есть надкатегории)
- [Численность врачей всех специальностей](https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-04.docx) (в таблице есть надкатегории)
- [Прерывание беременности (аборты)](https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-07.docx)  (в таблице есть надкатегории)
- [Соотношение мужчин и женщин](https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-04.docx)  (простая таблица)
- [Соотношение браков и разводов](https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-18.docx) (простая таблица)

Попробуем спарсить документ c надкатегориями:

In [474]:
parse_docx('06-01.docx')

6.1. ЧИСЛО БОЛЬНИЧНЫХ КОЕК1)
{'На 10 000 человек населения', 'Всего, тыс. коек'}

1) . – включая койки дневных стационаров.


Теперь загружаем полученные датафреймы:

In [475]:
beds_per_10k = pd.read_csv('20200901-091041a.csv')
beds_total = pd.read_csv('20200901-091041b.csv')

Посмотрим, что получилось:

In [476]:
beds_per_10k.head()

,Unnamed: 0,0,8,9,10,11,12,13,14,15
0,0,NaN,На 10 000 человек населения,На 10 000 человек населения,На 10 000 человек населения,На 10 000 человек населения,На 10 000 человек населения,На 10 000 человек населения,На 10 000 человек населения,На 10 000 человек населения
1,1,NaN,2005,2010,2014,2015,2016,2017,2018,"Место,\nзани-\nмаемое \nв Россий-\nской Феде-\..."
2,2,Российская Федерация,"110,9","93,8","86,6","83,4","81,6","80,5","79,9",NaN
3,3,Центральный \nфедеральный округ,"112,4","92,3","83,8","78,5","76,4","74,5","73,8",7
4,4,Белгородская область,"111,5","81,4","80,4","77,1","72,7","73,0","72,2",75


Я написала функцию, которая приводит данные к адекватному формату, при этом убирая мусорные строки и столбцы (подходит только для тех таблиц, где есть столбец "место в РФ" и "подвал"):

In [477]:
def clean_docx(df):
    df = df.iloc[1:-1, 1: -1] 
    df.columns = df.iloc[0]
    df = df[2:]
    df.columns = df.columns.fillna('region')
    df = pd.melt(df, id_vars = ['region'], var_name = 'year')
    df = df[~df['region'].str.contains('числе', na=False)]
    return df

Результат:

In [478]:
beds_per_10k = clean_docx(beds_per_10k)
beds_per_10k.head()

,region,year,value
0,Центральный \nфедеральный округ,2005,"112,4"
1,Белгородская область,2005,"111,5"
2,Брянская область,2005,"127,4"
3,Владимирская область,2005,"104,2"
4,Воронежская область,2005,"107,1"


Осталось разделить данные регионов и округов:

In [479]:
filter = beds_per_10k['region'].str.contains('федеральн', regex= True, na=False)

beds_per_10k_okrug = beds_per_10k[filter]
beds_per_10k_regions = beds_per_10k[~filter]

# подготовка к сохранению файлов
beds_per_10k_okrug.csv_path = 'beds_per_10k_okrug.csv'
beds_per_10k_regions.csv_path = 'beds_per_10k_regions.csv'

In [480]:
beds_per_10k_regions.head()

,region,year,value
1,Белгородская область,2005,"111,5"
2,Брянская область,2005,"127,4"
3,Владимирская область,2005,"104,2"
4,Воронежская область,2005,"107,1"
5,Ивановская область,2005,"125,7"


In [481]:
beds_per_10k_okrug.head()

,region,year,value
0,Центральный \nфедеральный округ,2005,"112,4"
19,Северо-Западный \nфедеральный округ,2005,"108,0"
33,Южный \nфедеральный округ,2005,"104,8"
42,Северо-Кавказский\nфедеральный округ,2005,"82,3"
50,Приволжский \nфедеральный округ,2005,"111,7"


То же самое необходимо проделать для таблицы с абсолютными значениями по больничным койкам:

In [482]:
beds_total.head()

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,NaN,"Всего, тыс. коек","Всего, тыс. коек","Всего, тыс. коек","Всего, тыс. коек","Всего, тыс. коек","Всего, тыс. коек","Всего, тыс. коек"
1,1,NaN,2005,2010,2014,2015,2016,2017,2018
2,2,Российская Федерация,"1575,4","1339,5","1266,8","1222,0","1197,2","1182,7","1172,8"
3,3,Центральный \nфедеральный округ,"428,2","354,7","326,3","306,8","299,5","292,7","290,5"
4,4,Белгородская область,"16,9","12,5","12,5","12,0","11,3","11,3","11,2"


In [483]:
beds_total = clean_docx(beds_total)
beds_total_okrug = beds_total[filter]
beds_total_regions = beds_total[~filter]

# подготовка к сохранению файлов
beds_total_okrug.csv_path = 'beds_total_okrug.csv'
beds_total_regions.csv_path = 'beds_total_regions.csv'

/Users/yuliyakuzmina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/yuliyakuzmina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [484]:
beds_total_regions.head()

,region,year,value
1,Белгородская область,2005,"16,9"
2,Брянская область,2005,"16,9"
3,Владимирская область,2005,"15,5"
4,Воронежская область,2005,"25,3"
5,Ивановская область,2005,"13,9"


Теперь спарсим данные о численности врачей всех специальностей:

In [485]:
parse_docx('06-04.docx')

6.4. ЧИСЛЕННОСТЬ ВРАЧЕЙ всех специальностей1)
(на конец года)
{'На 10 000 человек населения, человек', 'Всего, тыс. человек'}

1) С 2012, 2015 гг. – изменение методологии сбора данных Минздравом России.


In [486]:
docs_per_10k = pd.read_csv('20200901-105253a.csv')
docs_total = pd.read_csv('20200901-105253b.csv')

In [487]:
docs_per_10k.head() #проверим, что в этой таблице данные на 10 000 населения

,Unnamed: 0,0,8,9,10,11,12,13,14,15
0,0,NaN,"На 10 000 человек населения, человек","На 10 000 человек населения, человек","На 10 000 человек населения, человек","На 10 000 человек населения, человек","На 10 000 человек населения, человек","На 10 000 человек населения, человек","На 10 000 человек населения, человек","На 10 000 человек населения, человек"
1,1,NaN,2005,2010,2014,2015,2016,2017,2018,"Место,\nзанима- емое \nв Россий-\nской Феде-\n..."
2,2,Российская Федерация,"48,6","50,1","48,5","45,9","46,4","47,5","47,9",NaN
3,3,Центральный \nфедеральный округ,"50,9","53,5","50,2","46,1","46,3","46,8","47,7",4
4,4,Белгородская область,"39,6","40,4","40,7","41,3","41,6","41,1","40,5",69


In [488]:
docs_per_10k = clean_docx(docs_per_10k)
docs_total = clean_docx(docs_total)

In [489]:
filter = docs_per_10k['region'].str.contains('федеральн', regex= True, na=False)

docs_per_10k_okrug = docs_per_10k[filter]
docs_per_10k_regions = docs_per_10k[~filter]

# подготовка к сохранению файлов
docs_per_10k_okrug.csv_path = 'docs_per_10k_okrug.csv'
docs_per_10k_regions.csv_path = 'docs_per_10k_regions.csv'

In [490]:
docs_per_10k_regions.head()

,region,year,value
1,Белгородская область,2005,"39,6"
2,Брянская область,2005,"36,4"
3,Владимирская область,2005,"33,7"
4,Воронежская область,2005,"53,1"
5,Ивановская область,2005,"51,0"


In [491]:
filter = docs_total['region'].str.contains('федеральн', regex= True, na=False)

docs_total_okrug = docs_total[filter]
docs_total_regions = docs_total[~filter]

# подготовка к сохранению файлов
docs_total_okrug.csv_path = 'docs_total_okrug.csv'
docs_total_regions.csv_path = 'docs_total_regions.csv'

In [492]:
docs_total_okrug.head()

,region,year,value
0,Центральный \nфедеральный округ,2005,"193,9"
19,Северо-Западный \nфедеральный округ,2005,"74,5"
33,Южный \nфедеральный округ,2005,"60,3"
42,Северо-Кавказский\nфедеральный округ,2005,"33,5"
50,Приволжский \nфедеральный округ,2005,"142,1"


Показатель Аборты:

In [493]:
parse_docx('06-07.docx')

6.7. ПРЕРЫВАНИЕ БЕРЕМЕННОСТИ (АБОРТЫ)
{'На 100 родов', 'На 1000 женщин 15 – 49 лет'}
50


In [494]:
abortion_per100births = pd.read_csv('20200901-114932a.csv')
abortion_per1k_women = pd.read_csv('20200901-114932b.csv')

In [495]:
abortion_per100births = clean_docx(abortion_per100births)
abortion_per1k_women = clean_docx(abortion_per1k_women)

filter = abortion_per100births['region'].str.contains('федеральн', regex= True, na=False)
abortion_per100births_okrug = abortion_per100births[filter]
abortion_per100births_regions = abortion_per100births[~filter]

filter = abortion_per1k_women['region'].str.contains('федеральн', regex= True, na=False)
abortion_per1k_women_okrug = abortion_per1k_women[filter]
abortion_per1k_women_regions = abortion_per1k_women[~filter]

# подготовка к сохранению файлов
abortion_per100births_okrug.csv_path = 'abortion_per100births_okrug.csv'
abortion_per100births_regions.csv_path = 'abortion_per100births_regions.csv'

abortion_per1k_women_okrug.csv_path = 'abortion_per1k_women_okrug.csv'
abortion_per1k_women_regions.csv_path = 'abortion_per1k_women_regions.csv'

Теперь попробеум спарсить docx без "надепеременных", показатель Соотношение мужчин и женщин:

In [496]:
parse_docx('02-04.docx')

2.4. СООТНОШЕНИЕ МУЖЧИН И ЖЕНЩИН 
(оценка на конец года; на 1000 мужчин приходится женщин)
[]
970


In [497]:
gender_prop = pd.read_csv('20200901-110902.csv')

In [498]:
gender_prop.tail()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
94,94,Амурская область,1100.0,1109.0,1112.0,1113.0,1115.0,1108.0,1113.0,1114.0,1005.0,1113
95,95,Магаданская область,1042.0,1057.0,1060.0,1060.0,1062.0,1065.0,1065.0,1064.0,1066.0,1070
96,96,Сахалинская область,1078.0,1080.0,1077.0,1078.0,1077.0,1079.0,1081.0,1082.0,1075.0,1074
97,97,Еврейская автономная область,1088.0,1101.0,1103.0,1102.0,1105.0,1105.0,1108.0,1110.0,1110.0,1107
98,98,Чукотский автономный округ,939.0,995.0,971.0,966.0,964.0,961.0,965.0,963.0,967.0,970


В этой таблице нет переменной "место в РФ" и пояснений, поэтому я создала еще одну функцию:

In [499]:
def clean_easy_docx(df):
    df = df.iloc[:, 1: ]
    df.columns = df.iloc[0]
    df = df[2:]
    df.columns = df.columns.fillna('region')
    df = df[~df['region'].str.contains('числе', na=False)]
    df = pd.melt(df, id_vars = ['region'], var_name = 'year')
    return df

In [500]:
gender_prop = clean_easy_docx(gender_prop)
gender_prop.head()

,region,year,value
0,Центральный \nфедеральный округ,2005,1183
1,Белгородская область,2005,1177
2,Брянская область,2005,1191
3,Владимирская область,2005,1230
4,Воронежская область,2005,1191


In [501]:
filter = gender_prop['region'].str.contains('федеральн', regex= True, na=False)

gender_prop_okrug = gender_prop[filter]
gender_prop_regions = gender_prop[~filter]

# подготовка к сохранению файлов
gender_prop_okrug.csv_path = 'gender_prop_okrug.csv'
gender_prop_regions.csv_path = 'gender_prop_regions.csv'

In [502]:
gender_prop_regions.head()

,region,year,value
1,Белгородская область,2005,1177
2,Брянская область,2005,1191
3,Владимирская область,2005,1230
4,Воронежская область,2005,1191
5,Ивановская область,2005,1248


Скрипт будет работать и для других "простых" таблиц, например, для показателя Соотношение браков и разводов:


In [503]:
parse_docx('02-18.docx')

2.18. СООТНОШЕНИЕ БРАКОВ И РАЗВОДОВ
[]
671


In [504]:
mariage_divorce_prop = pd.read_csv('20200901-111606.csv')

mariage_divorce_prop = clean_easy_docx(mariage_divorce_prop)

filter = mariage_divorce_prop['region'].str.contains('федеральн', regex= True, na=False)
mariage_divorce_prop_okrug = mariage_divorce_prop[filter]
mariage_divorce_prop_regions = mariage_divorce_prop[~filter]

# подготовка к сохранению файлов
mariage_divorce_prop_okrug.csv_path = 'mariage_divorce_prop_okrug.csv'
mariage_divorce_prop_regions.csv_path = 'mariage_divorce_prop_regions.csv'

mariage_divorce_prop_regions.head()

,region,year,value
1,Белгородская область,2005,503.0
2,Брянская область,2005,575.0
3,Владимирская область,2005,570.0
4,Воронежская область,2005,629.0
5,Ивановская область,2005,646.0


Наконец, сохраняем полученные датасеты:

In [505]:
dfs = [beds_per_10k_okrug, beds_per_10k_regions, \
       docs_per_10k_okrug, docs_per_10k_regions, \
       abortion_per100births_okrug, abortion_per100births_regions, \
       abortion_per1k_women_okrug, abortion_per1k_women_regions, \
       gender_prop_okrug, gender_prop_regions, \
       mariage_divorce_prop_okrug, mariage_divorce_prop_regions]

for df in dfs:
    df.to_csv(df.csv_path)